In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('linear-regression').getOrCreate()

22/01/04 16:53:19 WARN Utils: Your hostname, alphatron resolves to a loopback address: 127.0.1.1; using 172.20.123.7 instead (on interface eth0)
22/01/04 16:53:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/04 16:53:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
df = spark.read.csv('advertising.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3| 12.0|
|151.5| 41.3|     58.5| 16.5|
|180.8| 10.8|     58.4| 17.9|
|  8.7| 48.9|     75.0|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|199.8|  2.6|     21.2| 15.6|
| 66.1|  5.8|     24.2| 12.6|
|214.7| 24.0|      4.0| 17.4|
| 23.8| 35.1|     65.9|  9.2|
| 97.5|  7.6|      7.2| 13.7|
|204.1| 32.9|     46.0| 19.0|
|195.4| 47.7|     52.9| 22.4|
| 67.8| 36.6|    114.0| 12.5|
|281.4| 39.6|     55.8| 24.4|
| 69.2| 20.5|     18.3| 11.3|
|147.3| 23.9|     19.1| 14.6|
+-----+-----+---------+-----+
only showing top 20 rows



In [5]:
df.printSchema()

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [6]:
# In PySpark we create a new feature out of all the
# independent features
# [TV, Radio, Newspaper] --> one feature

In [7]:
from pyspark.ml.feature import VectorAssembler

featureAssembler = VectorAssembler(inputCols=['TV', 'Radio', 'Newspaper'], outputCol='Independent Feature')

In [8]:
output = featureAssembler.transform(df)

In [9]:
output.show()

+-----+-----+---------+-----+-------------------+
|   TV|Radio|Newspaper|Sales|Independent Feature|
+-----+-----+---------+-----+-------------------+
|230.1| 37.8|     69.2| 22.1|  [230.1,37.8,69.2]|
| 44.5| 39.3|     45.1| 10.4|   [44.5,39.3,45.1]|
| 17.2| 45.9|     69.3| 12.0|   [17.2,45.9,69.3]|
|151.5| 41.3|     58.5| 16.5|  [151.5,41.3,58.5]|
|180.8| 10.8|     58.4| 17.9|  [180.8,10.8,58.4]|
|  8.7| 48.9|     75.0|  7.2|    [8.7,48.9,75.0]|
| 57.5| 32.8|     23.5| 11.8|   [57.5,32.8,23.5]|
|120.2| 19.6|     11.6| 13.2|  [120.2,19.6,11.6]|
|  8.6|  2.1|      1.0|  4.8|      [8.6,2.1,1.0]|
|199.8|  2.6|     21.2| 15.6|   [199.8,2.6,21.2]|
| 66.1|  5.8|     24.2| 12.6|    [66.1,5.8,24.2]|
|214.7| 24.0|      4.0| 17.4|   [214.7,24.0,4.0]|
| 23.8| 35.1|     65.9|  9.2|   [23.8,35.1,65.9]|
| 97.5|  7.6|      7.2| 13.7|     [97.5,7.6,7.2]|
|204.1| 32.9|     46.0| 19.0|  [204.1,32.9,46.0]|
|195.4| 47.7|     52.9| 22.4|  [195.4,47.7,52.9]|
| 67.8| 36.6|    114.0| 12.5|  [67.8,36.6,114.0]|


In [10]:
df = output.select('Independent Feature', 'Sales')

In [11]:
df.show()

+-------------------+-----+
|Independent Feature|Sales|
+-------------------+-----+
|  [230.1,37.8,69.2]| 22.1|
|   [44.5,39.3,45.1]| 10.4|
|   [17.2,45.9,69.3]| 12.0|
|  [151.5,41.3,58.5]| 16.5|
|  [180.8,10.8,58.4]| 17.9|
|    [8.7,48.9,75.0]|  7.2|
|   [57.5,32.8,23.5]| 11.8|
|  [120.2,19.6,11.6]| 13.2|
|      [8.6,2.1,1.0]|  4.8|
|   [199.8,2.6,21.2]| 15.6|
|    [66.1,5.8,24.2]| 12.6|
|   [214.7,24.0,4.0]| 17.4|
|   [23.8,35.1,65.9]|  9.2|
|     [97.5,7.6,7.2]| 13.7|
|  [204.1,32.9,46.0]| 19.0|
|  [195.4,47.7,52.9]| 22.4|
|  [67.8,36.6,114.0]| 12.5|
|  [281.4,39.6,55.8]| 24.4|
|   [69.2,20.5,18.3]| 11.3|
|  [147.3,23.9,19.1]| 14.6|
+-------------------+-----+
only showing top 20 rows



In [12]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = df.randomSplit([0.75, 0.25])

In [13]:
train_data.show()

+-------------------+-----+
|Independent Feature|Sales|
+-------------------+-----+
|     [4.1,11.6,5.7]|  3.2|
|     [5.4,29.9,9.4]|  5.3|
|    [7.3,28.1,41.4]|  5.5|
|    [7.8,38.9,50.6]|  6.6|
|     [8.4,27.2,2.1]|  5.7|
|      [8.6,2.1,1.0]|  4.8|
|    [8.7,48.9,75.0]|  7.2|
|   [13.2,15.9,49.6]|  5.6|
|    [17.2,4.1,31.6]|  5.9|
|   [17.9,37.6,21.6]|  8.0|
|   [18.7,12.1,23.4]|  6.7|
|   [19.6,20.1,17.0]|  7.6|
|   [23.8,35.1,65.9]|  9.2|
|   [25.0,11.0,29.7]|  7.2|
|   [25.1,25.7,43.3]|  8.5|
|    [28.6,1.5,33.0]|  7.3|
|    [31.5,24.6,2.2]| 11.0|
|   [36.9,38.6,65.6]| 10.8|
|    [38.2,3.7,13.8]|  7.6|
|   [43.0,25.9,20.5]|  9.6|
+-------------------+-----+
only showing top 20 rows



In [14]:
test_data.show()

+-------------------+-----+
|Independent Feature|Sales|
+-------------------+-----+
|     [0.7,39.6,8.7]|  1.6|
|   [11.7,36.9,45.2]|  7.3|
|    [13.1,0.4,25.6]|  5.3|
|   [16.9,43.7,89.4]|  8.7|
|   [17.2,45.9,69.3]| 12.0|
|   [18.8,21.7,50.4]|  7.0|
|   [19.4,16.0,22.3]|  6.6|
|    [25.6,39.0,9.3]|  9.5|
|   [26.8,33.0,19.3]|  8.8|
|    [27.5,1.6,20.7]|  6.9|
|   [38.0,40.3,11.9]| 10.9|
|    [39.5,41.1,5.8]| 10.8|
|   [43.1,26.7,35.1]| 10.1|
|    [48.3,47.0,8.5]| 11.6|
|   [66.9,11.7,36.8]|  9.7|
|     [69.0,9.3,0.9]| 11.3|
|     [80.2,0.0,9.2]| 11.9|
|    [89.7,9.9,35.7]| 10.6|
|     [94.2,4.9,8.1]| 14.0|
|     [95.7,1.4,7.4]| 11.9|
+-------------------+-----+
only showing top 20 rows



In [15]:
regressor = LinearRegression(featuresCol='Independent Feature', labelCol='Sales')

In [17]:
regressor = regressor.fit(train_data)

22/01/04 17:06:24 WARN Instrumentation: [2cbe4be9] regParam is zero, which might cause numerical instability and overfitting.


In [18]:
regressor.coefficients

DenseVector([0.0539, 0.1112, -0.0026])

In [19]:
test_output = regressor.evaluate(test_data)

In [21]:
test_output.predictions.show()

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------------+-----+------------------+
|Independent Feature|Sales|        prediction|
+-------------------+-----+------------------+
|     [0.7,39.6,8.7]|  1.6| 9.170616363528747|
|   [11.7,36.9,45.2]|  7.3| 9.367869499880062|
|    [13.1,0.4,25.6]|  5.3| 5.436256068071204|
|   [16.9,43.7,89.4]|  8.7| 10.28873490745773|
|   [17.2,45.9,69.3]| 12.0| 10.60197978207611|
|   [18.8,21.7,50.4]|  7.0| 8.046881107543218|
|   [19.4,16.0,22.3]|  6.6| 7.518821970816424|
|    [25.6,39.0,9.3]|  9.5|10.444085430273716|
|   [26.8,33.0,19.3]|  8.8| 9.815536416012545|
|    [27.5,1.6,20.7]|  6.9| 6.358418720766618|
|   [38.0,40.3,11.9]| 10.9| 11.25001584615341|
|    [39.5,41.1,5.8]| 10.8| 11.43571622154527|
|   [43.1,26.7,35.1]| 10.1| 9.952159602708166|
|    [48.3,47.0,8.5]| 11.6|12.558845330320967|
|   [66.9,11.7,36.8]|  9.7| 9.562431165227604|
|     [69.0,9.3,0.9]| 11.3| 9.502470906958713|
|     [80.2,0.0,9.2]| 11.9| 9.050306078166791|
|    [89.7,9.9,35.7]| 10.6|10.593758054825196|
|     [94.2,4